In [1]:
texto = """
A Microsoft foi fundada em 4 de abril de 1975 por Bill Gates e Paul Allen, em Albuquerque, no estado do Novo México. O nome "Microsoft" é uma combinação das palavras "microcomputador" e "software", refletindo o foco da empresa em software para computadores pessoais.
O primeiro grande projeto da Microsoft foi a criação de um sistema operacional para o computador Altair 8800, um dos primeiros microcomputadores disponíveis comercialmente.
O sistema, denominado Altair BASIC, foi desenvolvido em parceria com a MITS (Micro Instrumentation and Telemetry Systems) e foi um marco inicial para a Microsoft.
Em 1980, a empresa firmou um contrato significativo com a IBM para fornecer o sistema operacional para o novo PC da IBM, o que levou à criação do MS-DOS. Esse contrato foi um ponto de virada para a Microsoft, impulsionando sua expansão e dominando o mercado de sistemas operacionais para PCs nos anos seguintes.
Com o sucesso do MS-DOS, a Microsoft se consolidou como líder no setor de software e, em 1985, lançou o Windows, um sistema operacional gráfico que viria a se tornar a base de sua supremacia no mercado de sistemas operacionais para desktop.
"""

### Dado um texto que extrair dados

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List

from langchain_core.utils.function_calling import convert_pydantic_to_openai_function


In [3]:
class Event(BaseModel):
    name: str = Field(description="Descrição do evento")
    date: str = Field(description="Data do evento no formato dd/mm/yyyy")

class EventList(BaseModel):
    events: List[Event] = Field(description="Lista de eventos encontrados no texto")
    
tool_get_events = convert_pydantic_to_openai_function(EventList)

tool_get_events

{'name': 'EventList',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'events': {'description': 'Lista de eventos encontrados no texto',
    'type': 'array',
    'items': {'type': 'object',
     'properties': {'name': {'description': 'Descrição do evento',
       'type': 'string'},
      'date': {'description': 'Data do evento no formato dd/mm/yyyy',
       'type': 'string'}},
     'required': ['name', 'date']}}},
  'required': ['events']}}

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_tools import JsonOutputToolsParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente que extrai eventos históricos de textos."
            "Para cada evento encontrado no texto, identifique:"
            "\n- Uma descrição breve e clara do evento."
            "\n- A data no formato dd/mm/yyyy."
            "\n\nSe a data não estiver no formato correto, ajuste conforme necessário."
            "\n\nRetorne a lista completa dos eventos encontrados.",
        ),
        (
            "user",
            "Texto:\n{texto}\n\n" "Liste todos os eventos com suas respectivas datas.",
        ),
    ]
)

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | chat.bind_tools([tool_get_events]) | JsonOutputToolsParser()

chain.invoke({"texto": texto})

[{'args': {'events': [{'name': 'Fundação da Microsoft', 'date': '04/04/1975'},
    {'name': 'Desenvolvimento do Altair BASIC', 'date': '01/01/1975'},
    {'name': 'Contrato com a IBM para fornecer o sistema operacional para o PC da IBM',
     'date': '01/01/1980'},
    {'name': 'Lançamento do Windows', 'date': '01/01/1985'}]},
  'type': 'EventList'}]

### Extrair dados da web

In [11]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader("https://www.techtudo.com.br/")

docs = loader.load()

docs


[Document(page_content=" TechTudo: tecnologia, celular, computador e games                top e últimas       Voltar     top e últimas    últimas do techtudo     notícias     comparativos     dicas e tutoriais     guias     listas     reviews     vídeos     tudo sobre     webstories       celulares e tablets       Voltar     celulares e tablets    notícias de celulares     celulares     tablets     telefonia     smartwatches     e-reader     android     iOS     fórum     --     iPhone 14     iphone 13     galaxy s22 ultra     moto g stylus     redmi note 10       jogos       Voltar     jogos    notícias de jogos     jogos casuais     jogos de esportes     jogos de terror     mais jogos       Voltar     mais jogos    jogos de ação     jogos arcade     jogos de aventura     jogos de corrida     jogos de estratégia     jogos de luta     jogos de raciocínio     jogos de rpg     jogos de tabuleiro     jogos educativos       playstation     xbox     nintendo     fórum     --     gta 5     ro